In [7]:
import requests
import pandas as pd
import urllib.request as Request
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import numpy as np
import datetime as dt

site = 'https://klse.i3investor.com/jsp/newshl.jsp'

In [8]:
def scrape_news_text(news_url):
 
    #Get HTML of the webpage associated with news_url
    news_html =Request.Request(news_url, headers={'User-Agent': 'Mozilla/5.0'})
    news_content = urlopen(news_html).read()

    #Convert news_url to a BeautifulSoup object
    news_soup = BeautifulSoup(news_content , 'html.parser')
 
    #Use news_soup to get the news header, published date and contents
    newsheader =[par.text for par in news_soup.find_all('h2')]
    newsheader = " ".join(re.findall(r"[a-zA-Z0-9]+",''.join(newsheader)))
    
    try:
        publishDate = [par.next_sibling for par in news_soup.find_all('b')]
        publishDate =  " ".join(re.findall(r"[a-zA-Z0-9]+",''.join(publishDate)))
    except TypeError:
        publishDate = ""
        
    paragraphs = [par.text for par in news_soup.find_all('div', {"id":"newscontent"})]
    news_text = '\n'.join(paragraphs)
    final = " ".join(re.findall(r"[a-zA-Z0-9]+",news_text.lower()))
 
    #Lastly, join all together as array
    return np.array([newsheader, publishDate, final])

In [9]:
#Generalized function to get all news-related articles
def get_news_urls(links_site, key):
    #scrape the html of the site
    resp = Request.Request(links_site, headers={'User-Agent': 'Mozilla/5.0'})
    html  = urlopen(resp).read()
 
    #convert html to BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')
 
    #get list of all links on webpage
    links = soup.find_all('a')
    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]
 
    #Filter the list of urls to just the news articles with 'key'
    all_news_urls = [url for url in urls if '/'+key+'/' in url]
    
    protocol = '{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(site))
    all_articles = []

    for news_url in all_news_urls:
        print('scrapping from '+ news_url)
        all_articles.append(scrape_news_text(protocol + news_url))

    return all_articles

In [12]:
output = pd.DataFrame(get_news_urls(site, "fdnews"), columns = ["News Header", "Publish Date", "News Content"])

scrapping from /servlets/fdnews/246875.jsp
scrapping from /servlets/fdnews/246874.jsp
scrapping from /servlets/fdnews/246873.jsp
scrapping from /servlets/fdnews/246871.jsp
scrapping from /servlets/fdnews/246870.jsp
scrapping from /servlets/fdnews/246869.jsp
scrapping from /servlets/fdnews/246867.jsp
scrapping from /servlets/fdnews/246866.jsp
scrapping from /servlets/fdnews/246865.jsp
scrapping from /servlets/fdnews/246868.jsp
scrapping from /servlets/fdnews/246864.jsp
scrapping from /servlets/fdnews/246863.jsp
scrapping from /servlets/fdnews/246862.jsp
scrapping from /servlets/fdnews/246860.jsp
scrapping from /servlets/fdnews/246859.jsp
scrapping from /servlets/fdnews/246858.jsp
scrapping from /servlets/fdnews/246857.jsp
scrapping from /servlets/fdnews/246856.jsp
scrapping from /servlets/fdnews/246854.jsp
scrapping from /servlets/fdnews/246853.jsp
scrapping from /servlets/fdnews/246852.jsp
scrapping from /servlets/fdnews/246851.jsp
scrapping from /servlets/fdnews/246850.jsp
scrapping f

In [13]:
output.head()
output.to_csv('C:\\Users\\Asus\\Desktop\\UM Master Data Science\\Data mining\\Extraction\\StockNews_' + '{0:%Y%m%d}'.format(dt.datetime.now()) + '.csv', index = False, date_format='%Y-%m-%d %H:%M:%S')

In [ ]:
#Ignore this part
def scrape_all_articles(ticker , upper_page_limit = 5):
 
    landing_site = 'http://www.nasdaq.com/symbol/' + ticker + '/news-headlines'
 
    all_news_urls = get_news_urls(landing_site)
 
    current_urls_list = all_news_urls.copy()
 
    index = 2
 
    '''Loop through each sequential page, scraping the links from each'''
    while (current_urls_list is not None) and (current_urls_list != []) and \
        (index <= upper_page_limit):
 
        '''Construct URL for page in loop based off index'''
        current_site = landing_site + '?page=' + str(index)
        current_urls_list = get_news_urls(current_site)
 
        '''Append current webpage's list of urls to all_news_urls'''
        all_news_urls = all_news_urls + current_urls_list
 
        index = index + 1
 
    all_news_urls = list(set(all_news_urls))
 
    '''Now, we have a list of urls, we need to actually scrape the text'''
    all_articles = [scrape_news_text(news_url) for news_url in all_news_urls]
 
    return all_articles

In [14]:
import feedparser
from bs4 import BeautifulSoup
import pandas as pd

In [62]:
feeds=['http://www.thestar.com.my/rss/business/business-news/','http://www.thestar.com.my/rss/business/investing/','http://www.thestar.com.my/rss/business/sme/']
#feed=feedparser.parse("http://www.thestar.com.my/rss/business/business-news/")
#feed=feedparser.parse("http://www.thestar.com.my/rss/business/investing/")


In [63]:
data=[]
for j in feeds:
    feed=feedparser.parse(j)
    for i in feed['items']:
        title=i.title
        description=i.description
        link=i.link
        pubDate=i.published
        data.append({'title':title,'description':description,'link':link,'pubdate':pubDate})
staronline=pd.DataFrame(data,columns=['title','description','link','pubdate'])

In [64]:
staronline

,title,description,link,pubdate
0,Williams seeks compensation using Malaysian GP...,BAKU: The struggling Williams Formula One team...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 07:18:23 +08:00"
1,World markets themes for the week ahead,Following are five big themes likely to domina...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 07:07:32 +08:00"
2,Oil sinks 3% as Trump pressures Opec to lower ...,NEW YORK: Oil prices fell 3% on Friday after U...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 06:58:49 +08:00"
3,Malaysian palm oil price near 1-month low as o...,KUALA LUMPUR: Malaysian palm oil futures fell ...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 06:50:37 +08:00"
4,US GDP data boosts global stocks; S&amp;P 500 ...,NEW YORK: Global stock markets rose and the S&...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 06:43:50 +08:00"
5,"After Hyflux’s fall, S’pore debt buyers are sc...",The collapse of Singapore’s water treatment fi...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 00:00:00 +08:00"
6,Can-One controls 95.88% of KJCF,PETALING JAYA: The saga between Can-One Bhd an...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 00:00:00 +08:00"
7,"In business or politics, opportunities arise o...","Last Sunday, I dragged my wife to Old Town, Pe...",http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 00:00:00 +08:00"
8,Maybank demands payment from Hyflux unit withi...,PETALING JAYA: Malayan Banking Bhd (Maybank) h...,http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 00:00:00 +08:00"
9,Eye On Stock - Pharmaniaga,"DESPITE making some healthy headway, Pharmania...",http://www.thestar.com.my/business/business-ne...,"Sat, 27 Apr 2019 00:00:00 +08:00"


In [65]:
staronline.to_csv('C:\\Users\\Asus\\Desktop\\UM Master Data Science\\Data mining\\Extraction\\Star_Online_' + '{0:%Y%m%d}'.format(dt.datetime.now()) + '.csv', index = False,mode='a', date_format='%Y-%m-%d %H:%M:%S')
